# Introduction to CVXPY for Optimization

This notebook is from 2024 by Zikai Xiong and Sean Lo. Revised in 2025 by Karl Zhu.

CVXPY is an open-source Python-embedded modeling language for convex optimization problems. It lets you express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

If you really hate using Julia, then CVXPY is the alternative in Python that can help you out. Other popular Python packages include PuLP (for MILPs) and Pyomo (linear, nonlinear, convex, mixed-integer, stochastic, and more...).

To make things less repetitive and more interesting, we will also use PDLP - a new method to solve LPs that can be extended to be solved using GPUs. (https://arxiv.org/abs/2311.12180)

The documentation for:
- CVXPY https://www.cvxpy.org/
- PuLP https://coin-or.github.io/pulp/
- Pyomo https://www.pyomo.org/

In [ ]:
# Step 1: Install necessary packages
#!pip install cvxpy
#!pip install numpy
#!pip install gurobipy
#!pip install ortools==9.7.2996 # (The version after 9.8 is not supported for cvxpy)
#!pip install "cvxpy[PDLP]"


  Obtaining dependency information for cvxpy from https://files.pythonhosted.org/packages/9d/b0/19028903198cff00a03031aeb7fd4753e4379951e4a48986ef7e3503ff53/cvxpy-1.6.0-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for osqp>=0.6.2 from https://files.pythonhosted.org/packages/cd/6d/0d17e8fa61809c125f97685d86e6cd6f7b1e745e01b8d3f96d783c8de41b/osqp-0.6.7.post3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for clarabel>=0.5.0 from https://files.pythonhosted.org/packages/8b/b9/e41f5316a2d4261c340d9fa6aa1694dd57d12cc45f1e5dfc5773d2b53d39/clarabel-0.9.0-cp37-abi3-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata
  Obtaining dependency information for scs>=3.2.4.post1 from https://files.pythonhosted.org/packages/1f/e4/b757b7926cc3355ba41ba747afb5e4d6c553d05be840ea25dc47b47b47b1/scs-3.2.7.post2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for qdldl from https://files.py

In [21]:
!pip install ortools==9.7.2996

  Obtaining dependency information for ortools==9.7.2996 from https://files.pythonhosted.org/packages/2d/99/05949d19d6d8d69a391933b65dfc68b8bd0ede492e6f59e8da6fb444d81d/ortools-9.7.2996-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 12.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ortools
    Found existing installation: ortools 9.11.4210
    Uninstalling ortools-9.11.4210:
      Successfully uninstalled ortools-9.11.4210


In [1]:
# Step 2: import cvxpy and numpy packages
import cvxpy as cp
import numpy as np

In [2]:
# Step 3: check installed solvers
print(cp.installed_solvers())

['CBC', 'CLARABEL', 'GLOP', 'GUROBI', 'OSQP', 'PDLP', 'SCIPY', 'SCS']


## Section 1: Linear Optimization Problem

Define the optimization variables

In [6]:
x = cp.Variable()
y = cp.Variable()

Define the objective function and constraints

In [7]:
objective = cp.Maximize(3*x + 2*y)
constraints = [2*x + y <= 10, x + 3*y <= 12, x >= 0, y >= 0]

Create the optimization problem

In [8]:
linear_problem = cp.Problem(objective, constraints)

Solve the problem (We use the solver Gurobi but you can also use other solvers. )

In [9]:
linear_problem.solve(solver=cp.GUROBI,verbose=True)

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 16 12:45:12 PM: Your problem has 2 variables, 4 constraints, and 0 parameters.
(CVXPY) Jan 16 12:45:12 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 12:45:12 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 12:45:12 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 12:45:12 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 12:45:12 PM: Compiling problem (target solver=GUROBI).
(CV

16.4

Display the results

In [10]:
print("Optimal value:", linear_problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)

Optimal value: 16.4
Optimal x: 3.5999999999999996
Optimal y: 2.8000000000000003


## Section 2: A First-Order LP Solver (PDLP)

In this section we solve a randomly generated "big" LP instance using cvxpy and different solvers.

The problem is 
$$
\min c^\top x, \ s.t. Ax \le b \ ,
$$
in which $A$ has $m$ rows and $n$ columns. 

The data is created as follows:

In [3]:
np.random.seed(123)
m = 5000
n = 3000

# Generate A, b, c so that (x0,y0) is an optimal primal-dual solution to the LP problem 
s0 = np.random.randn(m)
y0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ y0

# Define the LP using CVXPY
x = cp.Variable(n)
big_LP = cp.Problem(cp.Minimize(c.T@x), [A @ x <= b])

Solve the problem using the primal simplex method of GUROBI (about 700 seconds).

In [4]:
opt_val=big_LP.solve(solver=cp.GUROBI, verbose=True, Method=0)
solution=x.value
print(opt_val)
print(solution)

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 16 01:16:51 PM: Your problem has 3000 variables, 5000 constraints, and 0 parameters.
(CVXPY) Jan 16 01:16:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 01:16:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 01:16:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 01:16:51 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 01:16:51 PM: Compiling problem (target solver=GUROBI

/Users/karlzhu/anaconda3/lib/python3.11/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Solve the problem using the dual simplex method of GUROBI (about 800 seconds.)

In [13]:
# Solve the problem using the default GUROBI dual simplex method
opt_val=big_LP.solve(solver=cp.GUROBI, verbose=True, Method=1)

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 16 12:53:48 PM: Your problem has 3000 variables, 5000 constraints, and 0 parameters.
(CVXPY) Jan 16 12:53:48 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 12:53:48 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 12:53:48 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 12:53:48 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 12:53:48 PM: Using cached ASA map, for faster compil

Solve the problem using PDLP (a first-order method for LP).  

The PDLP is the primal-dual hybrid gradient (PDHG) method with some practical enhancements. 
See more details of the mathematical background in https://developers.google.com/optimization/lp/pdlp_math. 

In [5]:
opt_val=big_LP.solve(solver=cp.PDLP, verbose=True)

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Jan 16 01:17:03 PM: Your problem has 3000 variables, 5000 constraints, and 0 parameters.
(CVXPY) Jan 16 01:17:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 16 01:17:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 16 01:17:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 16 01:17:03 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 16 01:17:03 PM: Compiling problem (target solver=PDLP).

Problem stats before presolving and rescaling:
There are 3000 variables, 5000 constraints, and 15000000 constraint matrix nonzeros.
Absolute values of nonzero constraint matrix elements: largest=5.381140, smallest=0.000000, avg=0.798038
Constraint matrix, infinity norm: max(row & col)=5.381140, min_col=3.082680, min_row=2.955656
Constraint bounds statistics (max absolute value per row): largest=220.898111, smallest=0.012123, avg=43.719825, l2_norm=3867.939661
Absolute values of objective vector elements: largest=200.838012, smallest=0.034710, avg=40.066409, l2_norm=2732.719799
Gaps between variable upper and lower bounds: #finite=0 of 3000, largest=0.000000, smallest=0.000000, avg=nan
Problem stats after presolving and rescaling:
There are 3000 variables, 5000 constraints, and 15000000 constraint matrix nonzeros.
Absolute values of nonzero constraint matrix elements: largest=0.071478, smallest=0.000000, avg=0.012799
Constraint matrix, infinity norm: max(row & col)=0.071478, min_col=0.0

It requires no more than 100 seconds!!!